## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-30-08-52-10 +0000,bbc,Virgin set to challenge Eurostar on Channel Tu...,https://www.bbc.com/news/articles/cy8v5z15551o...
1,2025-10-30-08-49-00 +0000,wsj,Euro Rises Ahead of ECB Decision,https://www.wsj.com/finance/currencies/asian-c...
2,2025-10-30-08-49-00 +0000,wsj,"Japan’s Central Bank Holds Fire as Tariffs, Po...",https://www.wsj.com/articles/japans-central-ba...
3,2025-10-30-08-49-00 +0000,wsj,Number of U.K. Companies Hit by Critical Finan...,https://www.wsj.com/economy/central-banking/nu...
4,2025-10-30-08-48-09 +0000,cbc,5 more suspects arrested in connection with Lo...,https://www.cbc.ca/news/world/5-more-suspects-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
55,trump,49
74,new,30
284,will,19
37,melissa,17
36,hurricane,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
84,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...,129
89,2025-10-30-02-08-41 +0000,nypost,Trump orders Pentagon to ‘immediately’ restart...,https://nypost.com/2025/10/29/us-news/trump-or...,119
78,2025-10-30-03-22-48 +0000,wapo,Trump says South Korea will be able to build n...,https://www.washingtonpost.com/world/2025/10/2...,110
237,2025-10-29-20-09-08 +0000,wapo,"Trump rebukes Putin for testing new missiles, ...",https://www.washingtonpost.com/world/2025/10/2...,107
42,2025-10-30-07-33-52 +0000,wapo,Trump cuts tariffs on China after ‘warm’ meeti...,https://www.washingtonpost.com/politics/2025/1...,105


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
84,129,2025-10-30-02-28-18 +0000,nypost,Trump says ‘we have a deal’ with China’s Xi Ji...,https://nypost.com/2025/10/29/us-news/trump-ch...
286,76,2025-10-29-18-05-17 +0000,wapo,Report: Government shutdown will cost the econ...,https://www.washingtonpost.com/business/2025/1...
181,74,2025-10-29-21-43-47 +0000,nypost,UK paid $41M to shoot new ‘Knives Out’ Netflix...,https://nypost.com/2025/10/29/media/uk-paid-41...
215,63,2025-10-29-20-56-14 +0000,wapo,Hurricane Melissa hits Cuba; has killed at lea...,https://www.washingtonpost.com/weather/2025/10...
99,63,2025-10-30-01-32-00 +0000,wsj,Federal Reserve Chair Jerome Powell delivered ...,https://www.wsj.com/economy/central-banking/fe...
417,56,2025-10-29-10-00-00 +0000,latimes,"Paramount lays off 1,000 workers in first roun...",https://www.latimes.com/entertainment-arts/bus...
326,44,2025-10-29-16-04-58 +0000,bbc,"Israeli strikes in Gaza kill 104, health minis...",https://www.bbc.com/news/articles/cgjdy5eevn2o...
89,43,2025-10-30-02-08-41 +0000,nypost,Trump orders Pentagon to ‘immediately’ restart...,https://nypost.com/2025/10/29/us-news/trump-or...
273,43,2025-10-29-18-30-22 +0000,latimes,Leading AI company to ban kids from long chats...,https://www.latimes.com/business/story/2025-10...
83,43,2025-10-30-02-40-50 +0000,latimes,He said he was an immigration lawyer but raped...,https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
